In [1]:
import os
import sys
import ee
import geemap
import numpy as np
import tqdm
import cv2
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Test on NLCD discrete pixel values

In [8]:
lon = -95.43495
lat = 29.75765

Map = geemap.Map()
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# Filter the collection to the 2016 product.
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()

# Select the land cover band.
landcover_2019 = nlcd2019.select('landcover')

# Display land cover on the map.
Map.setCenter(lon, lat, 20)
Map.addLayer(landcover_2019, None, 'Landcover')
Map

Map(center=[29.75765, -95.43495], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [10]:
point = ee.Geometry.Point(lon, lat)
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
image = dataset.filter(ee.Filter.eq('system:index', '2019')).first()
info = image.reduceRegion(ee.Reducer.first(), point, scale=30).getInfo()
pixel_value = info['landcover']
print(pixel_value)

90


It works! Note that lon, lat value of 5 decimal points (0.00001 deg, or 1m) lead to accurate result

## Test on ASTER GED

Let's try Penn State station (PSU)

In [11]:
from omegaconf import DictConfig, OmegaConf
import hydra
from eval.surfrad import get_emis_at
config = OmegaConf.load('../config/surfrad.yaml')

In [12]:
lon = config.stations.PSU.Longitude
lat = config.stations.PSU.Latitude

In [13]:
vis_params = {
    'min': 0.9,   # Set the minimum emissivity value you expect to see
    'max': 1.0,   # Set the maximum emissivity value you expect to see
    'palette': ['blue', 'green', 'yellow', 'red']  # Define a color palette from low to high emissivity
}

Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.setCenter(lon, lat, 20)
image = ee.Image('NASA/ASTER_GED/AG100_003').select('emissivity_band10').multiply(0.001)
Map.addLayer(image, vis_params, 'emis')
Map

Map(center=[40.72012, -77.93085], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [15]:
point = ee.Geometry.Point(lon, lat)
image = ee.Image('NASA/ASTER_GED/AG100_003').multiply(0.001)
info = image.reduceRegion(ee.Reducer.first(), point, scale=1).getInfo()
emis_10 = info['emissivity_band10']
emis_11 = info['emissivity_band11']
emis_12 = info['emissivity_band12']
emis_13 = info['emissivity_band13']
emis_14 = info['emissivity_band14']
print(emis_10, emis_11, emis_12, emis_13, emis_14)

0.967 0.964 0.961 0.974 0.974


In [16]:
broadband_emis = get_emis_at(lon, lat)
print('broadband_emis = ', broadband_emis)

broadband_emis =  0.9716729999999999
